In [12]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn import neighbors
from sklearn.metrics import roc_curve, auc

In [18]:
from simulate import simulate_samples
from plots import plot_simulation, plot_supervised
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Setup global settings

In [14]:
# Define default values for sliders
default_values = {
    'n_cells': 10000,
    'frac_leukemic_AML1': 0.1,
    'frac_leukemic_AML2': 0.01,
    'frac_leukemic_AML3': 0.1,
    'mean_healthy_x': -1,
    'std_healthy_x': 0.5,
    'mean_healthy_y': -1,
    'std_healthy_y': 0.5,
    'dist_healthy_LAIP1': 3,
    'std_LAIP1_x': 0.5,
    'std_LAIP1_y': 0.5,
    'dist_healthy_LAIP2': 3,
    'std_LAIP2_x': 0.5,
    'std_LAIP2_y': 0.5
}

# Define a function to restore default values
def restore_defaults(button):
    n_cells_slider.value = default_values['n_cells']
    frac_leukemic_AML1_slider.value = default_values['frac_leukemic_AML1']
    frac_leukemic_AML2_slider.value = default_values['frac_leukemic_AML2']
    frac_leukemic_AML3_slider.value = default_values['frac_leukemic_AML3']
    mean_healthy_x_slider.value = default_values['mean_healthy_x']
    std_healthy_x_slider.value = default_values['std_healthy_x']
    mean_healthy_y_slider.value = default_values['mean_healthy_y']
    std_healthy_y_slider.value = default_values['std_healthy_y']
    dist_healthy_LAIP1_slider.value = default_values['dist_healthy_LAIP1']
    std_LAIP1_x_slider.value = default_values['std_LAIP1_x']
    std_LAIP1_y_slider.value = default_values['std_LAIP1_y']
    dist_healthy_LAIP2_slider.value = default_values['dist_healthy_LAIP2']
    std_LAIP2_x_slider.value = default_values['std_LAIP2_x']
    std_LAIP2_y_slider.value = default_values['std_LAIP2_y']

# Setup sliders

In [15]:
slider_layout = widgets.Layout(width='400px')

# Create sliders with default values
n_cells_slider = widgets.IntSlider(value=default_values['n_cells'], min=1, max=100000, step=1, description="Total cells", style={'description_width': 'initial'}, layout=slider_layout)
frac_leukemic_AML1_slider = widgets.FloatSlider(value=default_values['frac_leukemic_AML1'], min=0, max=1, step=0.01, description="Fraction leukemic cells in AML1", style={'description_width': 'initial'}, layout=slider_layout)
frac_leukemic_AML2_slider = widgets.FloatSlider(value=default_values['frac_leukemic_AML2'], min=0, max=1, step=0.01, description="Fraction leukemic cells in AML2", style={'description_width': 'initial'}, layout=slider_layout)
frac_leukemic_AML3_slider = widgets.FloatSlider(value=default_values['frac_leukemic_AML3'], min=0, max=1, step=0.01, description="Fraction leukemic cells in AML3", style={'description_width': 'initial'}, layout=slider_layout)

mean_healthy_x_slider = widgets.FloatSlider(value=default_values['mean_healthy_x'], min=-10, max=10, step=0.1, description="Mean of non-leukemic cluster (X-axis)", style={'description_width': 'initial'}, layout=slider_layout)
std_healthy_x_slider = widgets.FloatSlider(value=default_values['std_healthy_x'], min=0, max=3, step=0.1, description="Standard deviation of non-leukemic cluster (X-axis)", style={'description_width': 'initial'}, layout=slider_layout)
mean_healthy_y_slider = widgets.FloatSlider(value=default_values['mean_healthy_y'], min=-10, max=10, step=0.1, description="Mean of non-leukemic cluster (Y-axis)", style={'description_width': 'initial'}, layout=slider_layout)
std_healthy_y_slider = widgets.FloatSlider(value=default_values['std_healthy_y'], min=0, max=3, step=0.1, description="Standard deviation of non-leukemic cluster (Y-axis)", style={'description_width': 'initial'}, layout=slider_layout)

dist_healthy_LAIP1_slider = widgets.FloatSlider(value=default_values['dist_healthy_LAIP1'], min=0, max=5, step=0.1, description="Distance between LAIP1+ and non-leukemic clusters", style={'description_width': 'initial'}, layout=slider_layout)
std_LAIP1_x_slider = widgets.FloatSlider(value=default_values['std_LAIP1_x'], min=0, max=3, step=0.1, description="Standard deviation of LAIP1+ cluster (X-axis)", style={'description_width': 'initial'}, layout=slider_layout)
std_LAIP1_y_slider = widgets.FloatSlider(value=default_values['std_LAIP1_y'], min=0, max=3, step=0.1, description="Standard deviation of LAIP1+ cluster (Y-axis)", style={'description_width': 'initial'}, layout=slider_layout)

dist_healthy_LAIP2_slider = widgets.FloatSlider(value=default_values['dist_healthy_LAIP2'], min=0, max=5, step=0.1, description="Distance between LAIP2+ and non-leukemic clusters", style={'description_width': 'initial'}, layout=slider_layout)
std_LAIP2_x_slider = widgets.FloatSlider(value=default_values['std_LAIP2_x'], min=0, max=3, step=0.1, description="Standard deviation of LAIP2+ cluster (X-axis)", style={'description_width': 'initial'}, layout=slider_layout)
std_LAIP2_y_slider = widgets.FloatSlider(value=default_values['std_LAIP2_y'], min=0, max=3, step=0.1, description="Standard deviation of LAIP2+ cluster (Y-axis)", style={'description_width': 'initial'}, layout=slider_layout)

In [10]:
# Create a button to restore defaults
restore_defaults_button = widgets.Button(description="Restore Defaults", button_style='primary')
restore_defaults_button.on_click(restore_defaults)

# Define the interactive plot with sliders
interactive_plot = interactive(plot_supervised,
                               n_cells = n_cells_slider,
                               frac_leukemic_AML1 = frac_leukemic_AML1_slider,
                               frac_leukemic_AML2 = frac_leukemic_AML2_slider,
                               frac_leukemic_AML3 = frac_leukemic_AML3_slider,
                               mean_healthy_x = mean_healthy_x_slider,
                               std_healthy_x = std_healthy_x_slider,
                               mean_healthy_y = mean_healthy_y_slider,
                               std_healthy_y = std_healthy_y_slider,
                               dist_healthy_LAIP1 = dist_healthy_LAIP1_slider,
                               std_LAIP1_x = std_LAIP1_x_slider,
                               std_LAIP1_y = std_LAIP1_y_slider,
                               dist_healthy_LAIP2 = dist_healthy_LAIP2_slider,
                               std_LAIP2_x = std_LAIP2_x_slider,
                               std_LAIP2_y = std_LAIP2_y_slider)

# Display the interactive plot and the restore defaults button
display(interactive_plot, restore_defaults_button)

interactive(children=(IntSlider(value=10000, description='Total cells', layout=Layout(width='400px'), max=1000…

Button(button_style='primary', description='Restore Defaults', style=ButtonStyle())

In [21]:
# Create a button to restore defaults
restore_defaults_button = widgets.Button(description="Restore Defaults", button_style='primary')
restore_defaults_button.on_click(restore_defaults)

# Define the interactive plot with sliders
interactive_plot = interactive(plot_simulation,
                               n_cells = n_cells_slider,
                               frac_leukemic_AML1 = frac_leukemic_AML1_slider,
                               frac_leukemic_AML2 = frac_leukemic_AML2_slider,
                               frac_leukemic_AML3 = frac_leukemic_AML3_slider,
                               mean_healthy_x = mean_healthy_x_slider,
                               std_healthy_x = std_healthy_x_slider,
                               mean_healthy_y = mean_healthy_y_slider,
                               std_healthy_y = std_healthy_y_slider,
                               dist_healthy_LAIP1 = dist_healthy_LAIP1_slider,
                               std_LAIP1_x = std_LAIP1_x_slider,
                               std_LAIP1_y = std_LAIP1_y_slider,
                               dist_healthy_LAIP2 = dist_healthy_LAIP2_slider,
                               std_LAIP2_x = std_LAIP2_x_slider,
                               std_LAIP2_y = std_LAIP2_y_slider)

# Display the interactive plot and the restore defaults button
display(interactive_plot, restore_defaults_button)

interactive(children=(IntSlider(value=10000, description='Total cells', layout=Layout(width='400px'), max=1000…

Button(button_style='primary', description='Restore Defaults', style=ButtonStyle())